# <font class="markdown-google-sans" size=7>**COQUI 🐸**</font>

> *Compact implementation of [**`camenduru/coqui-XTTS-colab`**](camenduru/coqui-XTTS-colab).*

<br><hr><br>

### Features:
- URL support 🔗
- **Batch processing** 🔰

<br><hr><br>

<a href="https://github.com/camenduru/coqui-XTTS-colab"><img src="https://img.shields.io/badge/github-open-lightgray?logo=github&logoColor=white&style=for-the-badge"></a>

# <div class="markdown-google-sans"><font color=lime>**1.**</font> Re/install requirements 📥 `≈03m56s`</div>

In [ ]:
#@markdown > ℹ️ Runtime does not need to be restarted.
import os
import re
import time
import torch
import locale
import warnings
import torchaudio
from time import time
from pathlib import Path
from shutil import rmtree
from zipfile import ZipFile
from datetime import timedelta
from rich import print as rprint
from mimetypes import guess_type
from psutil import virtual_memory
from requests import Session, utils
from contextlib import redirect_stdout
from IPython.display import clear_output

try:
	from google.colab import files, runtime
	is_colab = True
except ImportError:
	is_colab = False

GPU = torch.cuda.device_count()
warnings.filterwarnings("ignore")
locale.getpreferredencoding = lambda: "UTF-8"

#-=-=-=-#
# Metadata

__title__  = "Coqui XTTS"
__author__ = "kubinka0505"
__date__   = "31st December 2023"
__title__  = __title__.replace(" ", "_")

#-=-=-=-#
# Modules & packages

requirements = {
	"modules":  ["TTS==0.21.1", "langid", "unidic-lite", "unidic", "deepspeed", "mutagen"],
	"packages": ["sox"]
}

if GPU:
	GPU_Name = torch.cuda.get_device_name(torch.cuda.current_device())
	rprint("[b #00CC00]GPU Available![/] [#A0A0A0]({0})[/]".format(GPU_Name))
else:
	rprint("[#FF0000][b]GPU Unavailable!\n[/]Processing will be impossible - requirements will not be installed.[/]")
	raise SystemExit()

print("\n" + "─" * 32 + "\n")

try:
	__modules = requirements["modules"]
	__packages = requirements["packages"]

	if __modules:
		for Name in __modules:
			!pip uninstall {Name} -y -q | grep -v "WARNING: Skip"
			!pip install -U {Name} --progress-bar off | grep -v "already"
			if all((len(__modules) > 1, Name != __modules[-1])):
				print()

	print()

	!pip install numpy==1.22.0 --force-reinstall
	!python -m unidic download

	if __packages:
		for Name in __packages:
			!apt-get install --reinstall {Name} -qq > /dev/null 2 >&1
			if all((len(__packages) > 1, Name != __packages[-1])):
				print()

	import langid
	from TTS.api import TTS
	from TTS.tts.configs.xtts_config import XttsConfig
	from TTS.tts.models.xtts import Xtts
	from TTS.utils.manage import ModelManager
	from TTS.utils.generic_utils import get_user_data_dir

	from mutagen import File as mFile

	os.environ["COQUI_TOS_AGREED"] = "1"

	#-=-=-=-#
	# Requests

	session = Session()
	session.headers.update(
		{
			"User-Agent":
			"Mozilla/5.0 (Windows NT 10.0; Win32; x32) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
		}
	)

	#-=-=-=-#
	# Functions

	def fdb_size(obj: str, extended_units: bool = False, bits: bool = False, recursive: bool = False) -> str:
		"""
		Args:
			obj: Bytes integer or string path of existing file or directory.
			extended_units: Extends the unit of the result, i.e. "Megabytes" instead of "MB".
			bits: Uses decimal divider (1000) instead of binary one. (1024)
			recursive: Iterate subdirectories, applicable only if `obj` is directory. (slow!)

		Returns:
			Human-readable files size string.
		"""
		# Setup

		if bits:
			Bits = 1000
			Bits_Display_Single = "bit"
			Bits_Display_Multiple = "bits"
		else:
			Bits = 1024
			Bits_Display_Single = "byte"
			Bits_Display_Multiple = "bytes"

		Units = {
			"":	 "",
			"K": "kilo",
			"M": "mega",
			"G": "giga",
			"T": "tera",
			"P": "peta",
			"E": "exa",
			"Z": "zetta",
			"Y": "yotta",
			"R": "ronna",
			"Q": "quetta"
		}

		#-=-=-=-#
		# Search for files

		if isinstance(obj, str):
			path = str(Path(obj).resolve())

			if not os.path.exists(path):
				raise FileNotFoundError(path)

			if os.path.isfile(obj):
				Files = [obj]
			else:
				Wildcard = "*"
				if recursive:
					Wildcard += "*/*"
				Files = list(Path(obj).glob(Wildcard))
				for File in Files:
					Files[Files.index(File)] = str(File.resolve())

			Files = map(os.path.getsize, Files)
			Bytes = sum(Files)
		else:
			Bytes = obj

		#-=-=-=-#
		# Calculate integer

		for Unit in Units:
			if Bytes < Bits:
				break
			Bytes /= Bits

		#-=-=-=-#
		# Ending conditions

		if extended_units:
			if Bytes == 1:
				Bits_Display = Bits_Display_Single
			else:
				Bits_Display = Bits_Display_Multiple
			Unit = (Units[Unit] + Bits_Display).capitalize()
		else:
			Unit += "B"

		#-=-=-=-#
		# Add zero to integer

		if "." in str(Bytes):
			Bytes = round(Bytes, 2)
			Bytes = str(Bytes).split(".")
			Bytes[1] = Bytes[1][:2]
			Bytes[1] = Bytes[1].ljust(2, "0")
			Bytes = ".".join(Bytes)

		Bytes = str(Bytes)

		#-=-=-=-#
		# Return string

		return " ".join((Bytes, Unit))

	#-=-=-=-#
	# Directories

	Directory_Main = os.path.abspath(__title__)
	Directory_Primary = os.path.join(Directory_Main, "Repository")
	Directory_Inputs = os.path.join(Directory_Main,  "_Inputs")
	Directory_Outputs = os.path.join(Directory_Main, "_Outputs")

	for Directory in Directory_Inputs, Directory_Outputs:
		rmtree(Directory, True)
		os.makedirs(Directory, exist_ok = True)
except (KeyboardInterrupt, EOFError):
	print("\n" + "─" * 32 + "\n" + "Operation cancelled by user.")

# <div class="markdown-google-sans"><font color=gold>**2.**</font> Upload files 📤</font></div>

In [ ]:
#@markdown > 🗃️ <font color=lime>**<u>URL</u> of `ZIP` archive with files is supported.**

Direct_URL = "" #@param {type: "string"}
Allow_non_WAV_URL = True #@param {type: "boolean"}
__URL_Fail = False if Direct_URL else True

#@markdown ---

#@markdown > ℹ️ `Lazy Unpacking` determines whether <u>**files from the archive should only be unpacked if their file name extension**</u> is in the `Formats` variable.

Lazy_Unpacking = True #@param {type: "boolean"}
Formats = "AAC, AIFF, ALAC, APE, FLAC, M4A, MP3, OGG, OPUS, WAV, WV" #@param {type: "string"}
Formats = [Format.lower().strip() for Format in Formats.split(",")]

#@markdown ---

Ask_to_upload_files_if_download_succeeds = True #@param {type: "boolean"}
Remove_previously_uploaded_files_and_subdirectories = True #@param {type: "boolean"}

#-=-=-=-#

if "rmtree" not in locals():
	raise SystemExit("Requirements were not installed!\nPlease run the first cell.")

if "Files" not in locals():
	Files = []

#-=-=-=-#

__Directory_Base = os.getcwd()

# `os.chdir` since `files.upload` does not have the target directory argument
os.chdir(Directory_Inputs)

if Remove_previously_uploaded_files_and_subdirectories:
	Files_Inputs = []
	for File in Path(Directory_Inputs).rglob("*"):
		File = str(File.resolve())
		if is_colab:
			if ".ipynb_checkpoints" not in os.path.basename(File):
				Files_Inputs.append(File)
		else:
			File_Inputs.append(File)

	for File in Files_Inputs:
		if os.path.isfile(File):
			os.remove(File)
			rprint('Removed [b #44AAFF]"{0}"[/]'.format(
				os.path.relpath(File, Directory_Inputs)
			))

	for File in Files_Inputs:
		if os.path.isdir(File):
			rmtree(File)

	if Files_Inputs:
		print("\n" + "─" * 32 + "\n")

	Files = []

#-=-=-=-#

Allowed_Specifiers = "http", "https", "ftp"
Allowed_Specifiers = [Specifier + "://" for Specifier in Allowed_Specifiers]
Allowed_Specifiers = tuple(Allowed_Specifiers)

if Direct_URL.lower().startswith(Allowed_Specifiers):
	for Delimiter in (("?", "&", "#")):
		Direct_URL = Direct_URL.split(Delimiter)[0]

	Downloaded_File_Location = utils.unquote(Direct_URL.split("/")[-1])

	__URL_Fail = False
	try:
		Session_obj = session.get(Direct_URL, stream = 1)
	except Exception:
		__URL_Fail = True

	if __URL_Fail:
		rprint("[b #FF0000]Provided URL is invalid![/]")
		__URL_Fail = True

	if not __URL_Fail:
		with Session_obj as Site:
			if Site.ok:
				Site_MIME = Site.headers["content-type"].lower().split()[0].split(";")[0].split("/")
				if Site_MIME[0] == "audio":
					if any((Site_MIME[1].endswith("wav"), Allow_non_WAV_URL)):
						with open(Downloaded_File_Location, "wb") as File:
							rprint(f'Downloading [b #00CC00]"{Downloaded_File_Location}"[/]...')

							for Chunk in Site.iter_content(chunk_size = 1024):
								if Chunk:
									File.write(Chunk)
							Files.append(Downloaded_File_Location)
					else:
						rprint("[b #FF0000]Permission to download non-WAV files was not granted.[/]")
						__URL_Fail = True
				else:
					# ZIP Archive Handling
					if "/".join(Site_MIME) == "application/zip":
						with open(Downloaded_File_Location, "wb") as File:
							rprint(f'Downloading [b #00CC00]"{Downloaded_File_Location}"[/]...')

							for Chunk in Site.iter_content(chunk_size = 1024):
								if Chunk:
									File.write(Chunk)

						try:
							Archive_Files_Amount = 0
							with ZipFile(Downloaded_File_Location) as Archive:
								for Archive_File in Archive.infolist():
									if Lazy_Unpacking:
										if os.path.splitext(Archive_File.filename.lower())[1][1:] in Formats:
											Archive_Files_Amount += 1
											Archive.extract(Archive_File)
									else:
										Archive_Files_Amount += 1
										Archive.extract(Archive_File)

							os.remove(Downloaded_File_Location)

							#-=-=-=-#

							for File in Path().rglob("*.*"):
								File = str(File.resolve())

								if os.path.isfile(File):
									Files.append(File)

							rprint(f"[b #00CC00]{Archive_Files_Amount} files from main directory were extracted successfully![/]")
						except RuntimeError:
							rprint("[b #FF0000]Files extraction was not successful due to password requirement, file corruption or other internal error.[/]")
							__URL_Fail = True
					else:
						rprint("[b #FF0000]Provided URL is not an audio file.[/] [i #A0A0A0](undetected MIME type in header)[/]")
						__URL_Fail = True
			else:
				rprint("[b #FF0000]Failed to fetch the URL due to {0} status code. ({1})[/]".format(
					Site.status_code, Site.reason.title()
				))
				__URL_Fail = True

#-=-=-=-#
# Upload files

__Cancel_Permission = "" if __URL_Fail else " [i #A0A0A0](can be cancelled)[/]"

if any((Ask_to_upload_files_if_download_succeeds, __URL_Fail)):
	print("\n" + "─" * 32 + "\n")
	rprint(f"Upload [b #44AAFF]AUDIO[/] files{__Cancel_Permission}")

	try:
		for File in files.upload():
			Files.append(File)
	except TypeError:
		pass
	except (KeyboardInterrupt, EOFError):
		rprint("\n\n[b #FFCC00]File upload cancelled.[/]")

	Files = [str(Path(File).resolve()) for File in Files]
	if is_colab:
		Files = [File for File in Files if os.path.basename(File) != ".ipynb_checkpoints"]
	Files = list(set(Files))

#-=-=-=-#
# Check for files

Files = [os.path.abspath(File) for File in Files]

err = 0
try:
	# Use this later for string formatting
	Longest_File_Name_Length = [File.split(Directory_Inputs)[1][1:] for File in Files]
	Longest_File_Name_Length.sort(key = len)
	Longest_File_Name_Length = len(Longest_File_Name_Length[-1])
except IndexError:
	err = 1

if err:
	raise SystemExit("No uploaded files has been found.")

#-=-=-=-#

os.chdir(__Directory_Base)

In [ ]:
#@title <div class="markdown-google-sans">Verify files and convert them to `WAV` ⚙️</div>
#@markdown > ⚠️ **Files whose [MIME type](https://wikipedia.org/wiki/Media_type)'s first segment will not be detected as `audio` will be removed without warning.**
if "Files" not in locals():
	raise SystemExit("Requirements were not installed!\nPlease run the first cell.")

if not Files:
	raise SystemExit("No files has been found!")

#-=-=-=-#

Target = "WAV"

#-=-=-=-#

Target = Target.split()[0].upper().strip(".")

for Input in Files:
	Input_Name = os.path.basename(Input)

	try:
		Input_MIME = mFile(Input).mime
	except AttributeError:
		Input_MIME = guess_type(Input)
	Input_MIME = Input_MIME[0].lower().split("/")

	if Input_MIME[0] == "audio":
		if Input_MIME[1] != Target.lower():
			os.environ["FILE_INPUT"] = Input
			os.environ["FILE_OUTPUT"] = os.path.splitext(Input)[0] + f".{Target.lower()}"

			rprint(f'[b #00CC00]"{Input_Name}" is not a {Target} file, converting...[/]')
			!ffmpeg -i "$FILE_INPUT" -hide_banner -loglevel 16 \
				-map_metadata 0 -map_metadata 0:s:0 \
				-fflags +bitexact -flags:v +bitexact -flags:a +bitexact \
				"$FILE_OUTPUT"

			Files[Files.index(Input)] = os.environ["FILE_OUTPUT"]
			os.remove(Input)
	else:
		rprint(f'[b #FF0000]"{Input_Name}" is not an audio file, removing[/]')
		Files.remove(Input)
		os.remove(Input)

#-=-=-=-#

if not Files:
	raise SystemExit("No files has been found!")

# <div class="markdown-google-sans"><font color=gold>**3.**</font> Generate 🗣️</div>

In [ ]:
#@title <div class="markdown-google-sans">Setup `(≈02m19s)` ⚙️</div>
# For unknown reasons, takes over 6 minutes to install while put in the first cell

# Download XTTS
model_name = "tts_models/multilingual/multi-dataset/xtts_v2"
model_path = os.path.join(get_user_data_dir("tts"), model_name.replace("/", "--"))

config_json = os.path.join(model_path, "config.json")
vocab_json = os.path.join(model_path, "vocab.json")

if os.path.exists(model_path):
	rmtree(model_path)

# Download model
with redirect_stdout(None):
	ModelManager().download_model(model_name)

config = XttsConfig()
config.load_json(config_json)

# Loading model
model = Xtts.init_from_config(config)
model.load_checkpoint(
	config,
	checkpoint_path = os.path.join(model_path, "model.pth"),
	vocab_path = vocab_json,
	eval = True,
	use_deepspeed = True,
)
model.cuda()

supported_languages = config.languages

In [ ]:
#@title # <div class="markdown-google-sans">**Upload prompts file** 📝📤</div>

Prompts = """

# This line is ignored

# Please write your dialogs here

"""

#-=-=-=-#

Languages = {
	#"id":   ("long_name",  characters_limit),
	"en":    ["English",    250],
	"es":    ["Spanish",    239],
	"fr":    ["French",     273],
	"de":    ["Deutsch",    253],
	"it":    ["Italian",    213],
	"pt":    ["Portuguese", 203],
	"tr":    ["Turkish",    226],
	"ru":    ["Russian",    182],
	"nl":    ["Dutch",      251],
	"pl":    ["Polish",     224],
	"cs":    ["Czech",      186],
	"ar":    ["Arabic",     166],
	"zh-cn": ["Chinese",     82],
	"ja":    ["Japanese",    71],
	"ko":    ["Korean",      95],
	"hu":    ["Hungarian",  224]
}

#@markdown - <font color=lime>**Can be modified inside this cell as a `Prompts` multiline string variable**</font>

Upload_Prompt_File = True #@param {type: "boolean"}

#@markdown - Saved to `_prompt.txt`, inside main directory.
#@markdown - Separated by pipe: `|`
#@markdown
#@markdown ---

#@markdown ### Parameters:
#@markdown &emsp;<font color=red>`speaker_name`</font>`|`<font color=lime>`language_code`</font>`|`<font color=gold>`Some text`</font>`|`<font color=deepskyblue>`OUTPUT_NAME`</font>

#@markdown <br>

#@markdown <font color=red>`speaker_name`</font>: Existing file <u>without extension</u> inside `/content/Coqui_XTTS/_inputs` directory.

#@markdown <font color=lime>`language_code`</font>: Short language code like `en` (English), `de` (Deutsch) [etc](https://hf.co/coqui/XTTS-v2#languages).
#@markdown - *If `auto`, `none` or `x`, language is being recognized from prompt by the [`langid`](https://github.com/saffsd/langid.py) module.*

#@markdown <font color=gold>`Some text`</font>: A string shorter than the number of characters assigned to the language.
#@markdown - [[<u>**Languages character limits**</u>]](https://github.com/coqui-ai/TTS/blob/5dcc16d1931538e5bce7cb20c1986df371ee8cd6/TTS/tts/layers/xtts/tokenizer.py#L597-L614)
#@markdown - *The reason for the limit is models inability for longer text processing.*
#@markdown - *Colab prompt **display** is truncated to `75` characters.*

#@markdown <font color=deepskyblue>`OUTPUT_NAME`</font>: Output file name in `Directory_Outputs` directory.
#@markdown - *Optional, if not passed file is saved as <font color=gold>`Some_text`</font>`.wav`*
#@markdown - *Truncated to 150 characters by default.*

#@markdown ---

#@markdown ## Example file structure

#@markdown ```ini
#@markdown # Lines that are empty or that start with a #, like this one are ignored
#@markdown # Each element of list is stripped out of spaces - it can look like a table
#@markdown
#@markdown # 01. Dialogs of former/USA presidents alive in 2024
#@markdown biden     | x | Hi Barack and Donald    | HI_BIDEN
#@markdown obama     | x | Hi Joe and Donald       | HI_OBAMA
#@markdown # kennedy | x | I should not be here    | HI_KENNEDY
#@markdown # adams   | x | I was not even recorded | HI_ADAMS
#@markdown trump     | x | Hi Joe and Barrack      | HI_TRUMP
#@markdown
#@markdown # 02. (...)
#@markdown speaker_01|it|Ora so parlare italiano!
#@markdown speaker_02|en|I can speak in english
#@markdown speaker_01|de|I am experimenting
#@markdown
#@markdown # 03. (...)
#@markdown ```

#-=-=-=-#

IDs = {v[0]: k for k, v in Languages.items()}

if Upload_Prompt_File:
	fn = "_prompt.txt"
	fn = os.path.join(Directory_Inputs, fn)

	if os.path.exists(fn):
		os.remove(fn)

	err = 0
	try:
		files.upload_file(fn)

		with open(fn, "r", encoding = "UTF-8") as File:
			Prompts = File.read()
	except ValueError:
		err = 1

	if err:
		print()
		rprint(f"[b #FFCC00] File upload aborted, using prompts variable.[/]")
		print("\n" + "─" * 32)

#-=-=-=-#
# Old

Prompts_List = Prompts.strip("\n")
Prompts_List = Prompts_List.split("\n")
Prompts_List = [Line for Line in Prompts_List if Line]
__Lines_Old = len(Prompts_List)

# New
Prompts_List = [Line for Line in Prompts_List if not Line.startswith("#")]
__Lines_New = len(Prompts_List)

__Ignored = __Lines_Old - __Lines_New

print()
rprint("[b #{0}]Found {1} prompts, {2} {3} ignored.[/]".format(
	"00CC00" if __Lines_New else "FF0000",
	__Lines_New, __Ignored,
	"was" if __Ignored < 2 else "were"
))

In [ ]:
#@title # <div class="markdown-google-sans">**Run**</div>

def commaFix(s: str) -> str:
	return re.sub("([^\x00-\x7F]|\w)(\.|\。|\?)", r"\1 \2\2", s)

#@markdown > ## ⚠️ <font color=red> **Previously processed files with same speakers will be removed without warning!**</font>

#@markdown > ## 📉 <font color=gold>**Returns `24 000 Hz` mono files.**</font>
#@markdown > &nbsp;
#@markdown > #### *See also:* [**HiFi GAN BWE**](https://colab.research.google.com/github/kubinka0505/colab-notebooks/blob/master/Notebooks/AI/Audio/Upscale/HiFi_GAN_BWE.ipynb) ✨

#@markdown ---

#@markdown > *Performs `Prompt = re.sub("([^\x00-\x7F]|\w)(\.|\。|\?)", r"\1 \2\2", Prompt)` for each `Prompt`*

Use_Comma_Fix = True #@param {type: "boolean"}
Max_Prompt_Display_Length = 75
Truncate_Output_Name_After = 150

#@markdown ---

#@markdown > ### *Internal Coqui XTTS variables. Do not modify unless you are sure what you are doing.*
Length_GPT_Condition = 30 #@param {type: "integer"}
Length_GPT_Condition_Chunk = 4 #@param {type: "integer"}
Length_Reference_Max = 60 #@param {type: "integer"}

#@markdown > *If `Length_Reference_Max` is `0`, the seconds of <font color=red>`speaker_name`</font> file are taken.*

Repetition_Penalty = 5.0 #@param {type: "number"}
Temperature = 0.75 # @param {type:"slider", min:0.01, max:0.99, step:0.01}

#@markdown ---

#@markdown ### Other settings
Clear_Colab_console_output_after_each_prompt = False #@param {type: "boolean"}

#-=-=-=-#

if not "model" in locals():
	raise Exception("No model has been found! Please run the previous cell.")

#-=-=-=-#
# Preprocess prompts

Prompts_List = Prompts.strip("\n")
Prompts_List = Prompts_List.split("\n")

Prompts_List = [
	Line for Line in Prompts_List
	if not Line.startswith("#")
]

IDs = {v[0]: k for k, v in Languages.items()}
Prompts_List = [Line for Line in Prompts_List if Line]

#-=-=-=-#

Runtime = time()
Counter = 1

for Line in Prompts_List:
	Elements = Line.split("|")
	Elements = [Element.strip() for Element in Elements]

	#-=-=-=-#

	Speaker_Name = Elements[0]
	Language = Elements[1]
	Prompt_String = Elements[2]

	Speaker_Path = os.path.join(Directory_Inputs, Speaker_Name) + ".wav"
	Speaker_Display = os.path.splitext(os.path.basename(Speaker_Path))[0]

	if Language.lower().startswith(("auto", "none", "x")):
		Language = None

	if Use_Comma_Fix:
		Prompt_String = commaFix(Prompt_String)

	if Language:
		if Language in Languages:
			Language = IDs[Languages[Language][0]]
		else:
			raise ValueError("Unsupported language")
	else:
		Predicted = langid.classify(Prompt_String)[0].strip()

		if Predicted == "zh":
			Predicted = "zh-cn"

		if Language != Predicted:
			Language = Predicted

	MaxLen = Languages[Language][1]
	Language_Display = Languages[Language][0]

	#-=-=-=-#
	# Errors

	if os.path.basename(Speaker_Path) not in os.listdir(Directory_Inputs):
		raise FileNotFoundError(Speaker_Path)

	if len(Prompt_String) > MaxLen:
		rprint("[b #44AAFF]Prompt string length exceeds {}, truncating[/]")
		Prompt_String = Prompt_String[:MaxLen]

	if len(Prompt_String) < Max_Prompt_Display_Length:
		Prompt_Display = f'"{Prompt_String}"'
	else:
		Prompt_Display = f'"{Prompt_String[:Max_Prompt_Display_Length]}(...)"'

	#-=-=-=-#
	# Print information

	Counter_Display = str(Counter).rjust(len(str(len(Prompts_List))) + 1, "0")

	print(f"{Counter_Display}.")
	print("    Speaker Name: ", Speaker_Display)
	print("    Language:     ", "{0} ({1} characters limit)".format(Language_Display, MaxLen))
	print("    Prompt:       ", Prompt_Display)

	#-=-=-=-#
	# Setup output file name

	if len(Elements) > 3:
		output_name = Elements[3]
		rprint("    Output Name:  ", output_name)
	else:
		output_name = re.sub(r"[,.;@#?!&$]+\ *", "_", Prompt_String).replace("_", " ")
		output_name = output_name.split()
		output_name = "_".join([x for x in output_name if output_name])

	output_name = output_name[:Truncate_Output_Name_After]
	output_path = os.path.join(Directory_Outputs, Speaker_Display, output_name + ".wav")
	output_path = os.path.abspath(output_path)

	output_dir = os.path.dirname(output_path)

	if not os.path.exists(output_dir):
		os.makedirs(output_dir, exist_ok = True)

	if os.path.exists(output_path):
		os.remove(output_path)

	# Run
	if not Length_Reference_Max:
		Length_Reference_Max = round(mFile(Speaker_Path).info.length)

	torch.cuda.empty_cache()

	(gpt_cond_latent, speaker_embedding) = model.get_conditioning_latents(
		audio_path         = Speaker_Path,
		gpt_cond_len       = Length_GPT_Condition,
		gpt_cond_chunk_len = Length_GPT_Condition_Chunk,
		max_ref_length     = Length_Reference_Max
	)

	out = model.inference(
		Prompt_String, Language,
		gpt_cond_latent, speaker_embedding,
		repetition_penalty = Repetition_Penalty,
		temperature = Temperature,
	)

	torchaudio.save(
		output_path,
		torch.tensor(out["wav"]).unsqueeze(0),
		24000
	)

	if Clear_Colab_console_output_after_each_prompt:
		if Line != Prompts_List[-1]:
			clear_output(wait = True)
	else:
		print()

		rprint('[b][#44AAFF]\nSaved to "{0}" [#A0A0A0]({1}s) ({2})[/][/]'.format(
			os.path.relpath(output_path, Directory_Outputs),
			round(mFile(output_path).info.length, 3),
			fdb_size(output_path)
		))

		if Line != Prompts_List[-1]:
			print("\n" + "─" * 32 + "\n")

	Counter += 1

#-=-=-=-#

__Run_Time = time() - Runtime
print("\n" + "─" * 32 + "\n")
rprint("[b #00CC00]Done![/]")
rprint("Runtime: [b #44AAFF]{0}[/] [i #A0A0A0](avg. {1} per line)[/]".format(
	str(timedelta(seconds = __Run_Time))[2:-3],
	str(timedelta(seconds = __Run_Time / len(Prompts_List)))[2:-3]
))

# <div class="markdown-google-sans">Utils ⚙️</div>

In [ ]:
#@title <div class="markdown-google-sans">Convert output files to `FLAC` 🎶</div>

Bit_Depth = "24-bit" #@param ["16-bit", "24-bit"]
Bit_Depth = Bit_Depth.split("-")[0]
Normalize_dB = 1 # @param {type: "slider", min: -6, max: 1, step: 1}
#@markdown > ℹ️ To <font color=gold>**disable**</font> normalization, set `Normalize_dB` to **`1`**.

#-=-=-=-#

Target = "FLAC"
Target = Target.split()[0].lower().strip(".")

#-=-=-=-#

os.environ["BIT_DEPTH"] = "-b " + Bit_Depth
os.environ["NORMALIZE"] = ("--norm " + str(Normalize_dB)) if Normalize_dB < 1 else ""

for File in Path(Directory_Outputs).rglob("*.*"):
	Input = str(File.resolve())

	if os.path.isfile(Input):
		Input_MIME = mFile(Input).mime[0].lower().split("/")
		Input_Name = os.path.relpath(Input, Directory_Outputs)

		Output_FLAC = os.path.splitext(Input)[0], f".{Target.lower()}"
		Output_Temp = "_".join(Output_FLAC)
		Output_FLAC = "".join(Output_FLAC)

		os.environ["INPUT"] = Input
		os.environ["OUTPUT"] = Output_Temp

		rprint('[b i]{1}onverting [#44AAFF]"{0}"[/] to FLAC...[/]'.format(
			Input_Name,
			("rec" if Input_MIME[1] == "flac" else "c").capitalize()
		))

		!sox "$INPUT" -D $BIT_DEPTH -C 8 $NORMALIZE --comment "" -G "$OUTPUT" -V0

		os.remove(Input)
		os.rename(Output_Temp, Output_FLAC)

In [ ]:
#@title <div class="markdown-google-sans">Download archive with output files 🗃️</div>
Emphasize_archive_file_name = True #@param {type: "boolean"}
Beggining = "!_" if Emphasize_archive_file_name else ""

#@markdown > ℹ️ Pushes the archive file to top of the explorer's file list (**if** it's sorted by name) by putting `!_` to its file name beggining.

#-=-=-=-#

__Padding = 7
Archive_File = Beggining + "_".join((__title__, str(time()).split(".")[1][:__Padding].rjust(__Padding, "0")))
Archive_File += ".zip"
Archive_File = os.path.abspath(Archive_File)

#-=-=-=-#

# Archive Creation
print("Making Archive...")
with ZipFile(Archive_File, "w", 14, 1, 9) as Archive:
	for File in Path(Directory_Outputs).rglob("*.*"):
		File = str(File.resolve())

		if os.path.isfile(File):
			ArcName = os.path.relpath(File, Directory_Outputs)

			rprint(f'\t[b i]Writing [#44AACC]"{ArcName}"[/]...')
			Archive.write(File, ArcName)

#-=-=-=-#

files.download(Archive_File)

In [ ]:
#@title <font class="markdown-google-sans">Terminate runtime after specified time</font> 🔌
#@markdown In case if you don't want to waste this account's resources while performing other tasks.
#@markdown > ### <font color=red>**This <font color=gold>cancellable</font> action is irreversible and will result in loss of ALL session data!**</font>

#@markdown <br>
#@markdown <details open>
#@markdown     <summary>Time sheet</summary>
#@markdown
#@markdown <br>
#@markdown
#@markdown | Seconds | Minutes |
#@markdown |:-|:-|
#@markdown | `60` | `1` |
#@markdown | `300` | `5` |
#@markdown | `600` | `10` |
#@markdown | `1800` | `30` |
#@markdown | `3600` | `60` |
#@markdown </details>

#@markdown ---

Seconds_Default = 30
Seconds = 300 # @param {type: "integer"}

if Seconds < 0:
	rprint(f"[b][#FFCC00]Seconds are[/] [#FF0000]< 0[/][#FFCC00], defaulting to[/] [#FF0000]{Seconds_Default}[/][#FFCC00].[/][/]")

__Spaces = " " * 8



if is_colab:
	try:
		for i in range(Seconds, -1, -1):
			print("\r", end = f"Runtime will be terminated in {i} seconds..." + __Spaces)

		rprint("\n\n[b #FF0000]Runtime has been terminated.[/]")
		runtime.unassign()
	except (KeyboardInterrupt, EOFError):
		print()
		rprint("[b #FF0000]Operation cancelled by user.[/]")
else:
	rprint("[b #FFCC00]Not in Colab, unsupported.[/]")